In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Solve the imshow dead kernel problem
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
'''
Start loading the data
'''
print('================== START LOADING DATA ==================')

================== START LOADING DATA ==================


In [4]:
from PIL import Image
import os
import os.path

import torch.utils.data
import torchvision.transforms as transforms

import numpy as np
import pandas as pd

In [75]:
# Split the training set into a 80% training set and 20% validation set
import random

def split_huge_file(file,out1,out2,percentage=0.75,seed=2022):
    """Splits a file in 2 given the approximate `percentage` to go in the large file."""
    random.seed(seed)
    with open(file, 'r',encoding="utf-8") as fin, \
         open(out1, 'w') as foutBig, \
         open(out2, 'w') as foutSmall:

        for line in fin:
            r = random.random() 
            if r < percentage:
                foutBig.write(line)
            else:
                foutSmall.write(line)

In [76]:
path = '/content/drive/My Drive/'
split_huge_file(os.path.join(path, f'train_triplets.txt'), 'train_triplets_splits.txt', 'val_triplets_splits.txt', percentage=0.99, seed=2022)

In [78]:
torch.cuda.empty_cache()

In [79]:
!nvidia-smi

Wed May  4 09:36:14 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P0    31W /  70W |   2116MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [80]:
# Image loader helper function
def default_image_loader(path):
    return Image.open(path).convert('RGB')

In [81]:
# Dataset
im = Image.open(r"/content/drive/My Drive/food/00003.jpg")

In [82]:
# display(im)

In [83]:
data = np.asarray(im)
data.shape

(329, 468, 3)

In [84]:
# im.resize((354,242))

In [85]:
class TripletImageLoader(torch.utils.data.Dataset):
    def __init__(self, base_path, triplets_file_name, transform=None, loader=default_image_loader):
        """ base_path: The path contains the text file of the training triplets
            triplets_file_name: The text file with each line containing three integers, 
            where integer i refers to the i-th image in the filenames file.  
            Each line contains three integers (a triplet).
            For example, the triplet "00723 00478 02630" denotes that the dish in image "00723.jpg" is more similar in taste 
            to the dish in image "00478.jpg" than to the dish in image "02630.jpg" according to a human annotator.
         """
        self.base_path = base_path  
        triplets = []
        for line in open(triplets_file_name):
            triplets.append((line.split()[0], line.split()[1], line.split()[2])) # anchor, positive, negative
        self.triplets = triplets
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        path1, path2, path3 = self.triplets[index]
        img1 = self.loader(os.path.join(self.base_path, f'{path1}.jpg'))
        img2 = self.loader(os.path.join(self.base_path, f'{path2}.jpg'))
        img3 = self.loader(os.path.join(self.base_path, f'{path3}.jpg'))
        if self.transform is not None:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
            img3 = self.transform(img3)

        return img1, img2, img3

    def __len__(self):
        return len(self.triplets)

In [86]:
# Initialization: importing the packages that we will use
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu' # Google colab offers time limited use of GPU for free

################# Configuration  ######################
IMAGE_SIZE = (242, 354) # bigger image size improves performance but makes training slower.

# Training parameters 
BATCH_SIZE = 64

In [87]:
torch.cuda.is_available()

True

In [88]:
device

'cuda'

In [89]:
!cp /content/drive/MyDrive/test_triplets.txt /content

In [90]:
# Dataset and Trasformations
import torchvision
import torchvision.transforms as transforms

############# Datasets and Dataloaders ################
transform_train = transforms.Compose([
    transforms.ToTensor(), # The output of torchvision datasets are PILImage images of range [0, 1].
    transforms.Resize(IMAGE_SIZE),
    transforms.RandomHorizontalFlip(p=0.5), # we want our network to be robust over geometrical transformations that leave the image semantically invariant
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)), #  We transform them to Tensors of normalized range [-1, 1].
    # (mean, mean, mean) , (std, std, std): output[channel] = (input[channel] - mean[channel]) / std[channel]
])

transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(IMAGE_SIZE),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(IMAGE_SIZE),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

path = '/content/drive/MyDrive/food'
train_dataset = TripletImageLoader(path.rstrip('\n'), 'train_triplets_splits.txt', transform=transform_train)
val_dataset = TripletImageLoader(path.rstrip('\n'), 'val_triplets_splits.txt', transform=transform_val)
test_dataset = TripletImageLoader(path.rstrip('\n'), 'test_triplets.txt', transform=transform_test)

In [91]:
len(train_dataset), len(val_dataset), len(test_dataset)

(58933, 582, 59544)

In [92]:
test_dataset[0]

(tensor([[[ 0.6848,  0.6188,  0.6295,  ...,  0.6845,  0.6174,  0.5918],
          [ 0.7634,  0.6081,  0.6502,  ...,  0.7868,  0.7373,  0.6763],
          [ 0.7259,  0.7257,  0.6671,  ...,  0.7282,  0.7121,  0.6763],
          ...,
          [ 0.1368,  0.1779,  0.1858,  ...,  0.5491,  0.5174,  0.5192],
          [ 0.1719,  0.1433,  0.1679,  ...,  0.5418,  0.5576,  0.5111],
          [ 0.2298,  0.0134,  0.0636,  ...,  0.6480,  0.7295,  0.7176]],
 
         [[ 0.3083,  0.2423,  0.2557,  ...,  0.2424,  0.3027,  0.3446],
          [ 0.3778,  0.2235,  0.2622,  ...,  0.3537,  0.3888,  0.3711],
          [ 0.3208,  0.3215,  0.2648,  ...,  0.3097,  0.3167,  0.2928],
          ...,
          [-0.2550, -0.2139, -0.2107,  ...,  0.0953,  0.1322,  0.1737],
          [-0.2046, -0.2332, -0.2138,  ...,  0.0879,  0.1528,  0.1360],
          [-0.1141, -0.3449, -0.3147,  ...,  0.1256,  0.0728, -0.0351]],
 
         [[ 0.1512,  0.0762,  0.0626,  ..., -0.0076, -0.0635, -0.0872],
          [ 0.2222,  0.0503,

In [93]:
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)

val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [94]:
len(train_loader), len(val_loader), len(test_loader)

(921, 10, 931)

In [95]:
# Visualization of Dataset
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    plt.figure()
    plt.imshow(img.permute(1, 2, 0))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images_anchor, images_positive, images_negative = dataiter.next()

# # show images
# imshow(torchvision.utils.make_grid(images_anchor))
# imshow(torchvision.utils.make_grid(images_positive))
# imshow(torchvision.utils.make_grid(images_negative))

In [96]:
'''
Data loaded
'''
print('================== DATA LOADED ==================')

================== DATA LOADED ==================


In [97]:
'''
Start constructing the network
'''
print('================== START CONSTRUCTING NETWORK ==================')

================== START CONSTRUCTING NETWORK ==================


In [27]:
# import torch.optim
# import torch.utils.data
# import torch
# import torchvision
# import torch.nn as nn
# import torchvision.models as models
# import torch.utils.data
# import torch.backends.cudnn as cudnn

# #########################NET##############################

# #The backbone for the CNNS with shared weights
# def backbone(**kwargs):
#     """
#     Construct a ResNet-101 model.
#     Returns:
#         Embeddingnet(model): The CNN with the specified model as its backbone is instantiated
#     """
#     #model = torch.hub.load('pytorch/vision:v1.7.1', 'resnet101', pretrained=True)
#     model = models.resnet18(pretrained=True)
#     #model = models.resnet34(pretrained=True)
#     #model = models.vgg11_bn()
#     #model = torch.hub.load('pytorch/vision:v0.8.2', 'alexnet', pretrained=True)
#     #model = models.alexnet(pretrained=True)            #used in the paper
#     #print('Layers',model.children)
#     #model = models.resnet50(pretrained=True)
#     #model = models.inception_v3(pretrained=True)
#     #model = torchvision.models.resnet.ResNet(
#         #torchvision.models.resnet.BasicBlock, [2, 1, 1, 1])

#     return EmbeddingNet(model)

# #The overall network consisting of three embedding nets with shared weights
# class TripletNet(nn.Module):
#     """Triplet Network."""

#     def __init__(self, embeddingnet):
#         """Triplet Network Builder."""
#         super(TripletNet, self).__init__()
#         self.embeddingnet = embeddingnet
#         #print(self.embeddingnet.children())

#     def forward(self, a, p, n):
#         """Forward pass."""
#         # anchor
#         embedded_a = self.embeddingnet(a)

#         # positive examples
#         embedded_p = self.embeddingnet(p)

#         # negative examples
#         embedded_n = self.embeddingnet(n)

#         return embedded_a, embedded_p, embedded_n

# #The CNN used by Triplet Net with 'model' as its backbone and a final fully connected Layer
# class EmbeddingNet(nn.Module):
#     """EmbeddingNet using the specified model in backbone()."""

#     def __init__(self, resnet):
#         """Initialize EmbeddingNet model."""
#         super(EmbeddingNet, self).__init__()
#         # Everything excluding the last linear layer
#         self.features = nn.Sequential(*list(resnet.children())[:-1])
#         num_ftrs =  resnet.fc.in_features
#         self.fc1 = nn.Linear(num_ftrs, 1024)

#     def forward(self, x):
#         """Forward pass of EmbeddingNet."""
#         out = self.features(x)
#         out = out.view(out.size(0), -1)
#         out = self.fc1(out)
#         return out

In [98]:
# Construct a triplet net
import torch.optim
import torch.utils.data
import torch
import torchvision
import torch.nn as nn
import torchvision.models as models
import torch.utils.data
import torch.backends.cudnn as cudnn

#########################NET##############################

#The backbone for the CNNS with shared weights
def FeatureExtractNET(**kwargs):
    """
    Construct a ResNet-101 model.
    Returns: The CNN for feature extraction with a fully connected layer
    """
    model = models.resnet18(pretrained=True)

    return EmbeddingNet(model)

#The CNN used by Triplet Net with 'model' as its backbone and a final fully connected Layer
class EmbeddingNet(nn.Module):
    """EmbeddingNet using the specified model in backbone()."""

    def __init__(self, resnet):
        """Initialize EmbeddingNet model."""
        super(EmbeddingNet, self).__init__()
        # Everything excluding the last linear layer
        self.features = nn.Sequential(*list(resnet.children())[:-1])
        num_ftrs =  resnet.fc.in_features
        self.fc1 = nn.Linear(num_ftrs, 1024)

    def forward(self, x):
        """Forward pass of EmbeddingNet."""
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        return out
    
#The overall network consisting of three embedding nets with shared weights
class TripletNet(nn.Module):
    """Triplet Network."""

    def __init__(self, embeddingnet):
        """Triplet Network Builder."""
        super(TripletNet, self).__init__()
        self.embeddingnet = embeddingnet

    def forward(self, a, p, n):
        """Forward pass."""
        # anchor
        embedded_a = self.embeddingnet(a)

        # positive examples
        embedded_p = self.embeddingnet(p)

        # negative examples
        embedded_n = self.embeddingnet(n)

        return embedded_a, embedded_p, embedded_n

In [99]:
net = TripletNet(FeatureExtractNET())

#Move the net to GPU for training
print("==> Initialize CUDA support for TripletNet model ...")
net = torch.nn.DataParallel(net).cuda()
cudnn.benchmark = True
net

==> Initialize CUDA support for TripletNet model ...


DataParallel(
  (module): TripletNet(
    (embeddingnet): EmbeddingNet(
      (features): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (4): Sequential(
          (0): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inplace=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (1): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)

In [100]:
torch.cuda.empty_cache()

In [101]:
# batch = next(iter(train_loader))
# batch[0], batch[1], batch[2] = batch[0].cuda(), batch[1].cuda(), batch[2].cuda()
# net(batch[0],batch[1],batch[2])[0].size()

In [102]:
# import hiddenlayer as hl

# transforms = [hl.transforms.Prune('Constant')] # Removes Constant nodes from graph.

# graph = hl.build_graph(net, (batch[0], batch[1], batch[2]), transforms=transforms)
# graph.theme = hl.graph.THEMES['blue'].copy()
# graph.save('rnn_hiddenlayer_1', format='png')

In [103]:
import torch.optim as optim
criterion = nn.TripletMarginLoss(margin=5.0, p=2)

optimizer = optim.SGD(net.parameters(),
                            lr=0.0005,
                            momentum=0.9,
                            weight_decay=2e-3,#The value used in the paper is 1e-3
                            nesterov=True)

In [104]:
torch.cuda.empty_cache()

In [105]:
# from torch.autograd import Variable
# for epoch in range(1):

#         running_loss = 0.0
#         loss_train = 0.0
#         for batch_idx, (data1, data2, data3) in enumerate(train_loader):

# #             if is_gpu:
# #                 data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()

#             # wrap in torch.autograd.Variable
#             data1, data2, data3 = Variable(
#                 data1), Variable(data2), Variable(data3)
#             print('anchor', data1.size())
#             print('positive', data2.size())
#             print('negative', data3.size())

#             # compute output and loss
#             embedded_a, embedded_p, embedded_n = net(data1, data2, data3)
#             loss = criterion(embedded_a, embedded_p, embedded_n)
#             print(loss)

#             # compute gradient and do optimizer step
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             # print the loss
#             running_loss += loss.data

# #             loss_train_cls = torch.sum(
# #                 1 * (criterion_val(embedded_a, embedded_p,
# #                                    embedded_n) > 0)) / train_batch_size  # CHANGED, MAY NEED TO REVERT BACK

# #             loss_train += loss_train_cls.data

#             if batch_idx % 30 == 0:
#                 print("mini Batch Loss: {}".format(loss.data))

In [106]:
'''
Network constructed
'''
print('================== NETWORK CONSTRUCTED ==================')

================== NETWORK CONSTRUCTED ==================


In [107]:
# import torch
# import torch.nn as nn
# pdist = nn.PairwiseDistance(p=2)
# input1 = torch.randn(64, 1024)
# input2 = torch.randn(64, 1024)
# input3 = torch.randn(64, 1024)
# print(input1.size())
# print(input2.size())
# print(input3.size())
# dist1 = pdist(input1, input2)
# dist2 = pdist(input1, input3)
# print(dist1.size())
# print(dist2.size())
# pred = dist1 - dist2
# print(pred.size())
# sum = 0
# for i in range(pred.size()[0]):
#   if pred[i] < 0:
#     sum+=1
# print(sum/pred.size()[0])
# print((pred < 0).sum()*1.0/pred.size()[0])

In [108]:
# import random
# for i in range(10):
#   a = random.randint(0,10)
#   print(a)

In [109]:
import torch.nn.functional as F
from torch.autograd import Variable

def accuracy(dista, distb):
    margin = 0
    pred = (dista - distb).cpu().data
    return (pred < 0).sum()*1.0/dista.size()[0]

def val_accuracy(trainednet, valloader, valloader_iter):
  # sum_accuracy = 0
  # num_batch_evaluated = 0
  
  # Pick one batches for evaluation
  # selected_batch = random.randint(0, len(valloader))

  # print('Batch selected for evaluation: ', selected_batch)
  try:
    data1, data2, data3 = next(valloader_iter)
  except StopIteration:
    valloader_iterator = iter(valloader)
    data1, data2, data3 = next(valloader_iterator)

  data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()

  # wrap in torch.autograd.Variable
  data1, data2, data3 = Variable(data1), Variable(data2), Variable(data3)

  with torch.no_grad():
    # compute output and loss
    embedded_x, embedded_y, embedded_z = trainednet(data1, data2, data3)
    dist_a = F.pairwise_distance(embedded_x, embedded_y, 2)
    dist_b = F.pairwise_distance(embedded_x, embedded_z, 2)
    print('dist a: {0}, dist b: {1}'.format(dist_a, dist_b))
    batch_accuracy = accuracy(dist_a, dist_b)
    print('random batch accuracy: {0} '.format(batch_accuracy))

  # mean_accuracy = sum_accuracy / num_batch_evaluated
  return batch_accuracy

In [110]:
# torch.cuda.empty_cache()
# batch = next(iter(train_loader))
# batch[0], batch[1], batch[2] = batch[0].cuda(), batch[1].cuda(), batch[2].cuda()
# embedded_x, embedded_y, embedded_z = net(batch[0],batch[1],batch[2])
# dist_a = F.pairwise_distance(embedded_x, embedded_y, 2)
# dist_b = F.pairwise_distance(embedded_x, embedded_z, 2)
# accuracy = accuracy(dist_a, dist_b)
# print(accuracy)

In [117]:
def train(model, criterion, optimizer, epochs, trainloader, valloader):

  # Create an iterator object for valloader, for selecting a random batch from val set for validation
  valloader_iterator = iter(valloader)

  # Empty the cache of CUDA  
  torch.cuda.empty_cache()
  
  print('================== START TRAINING ==================')
  # Change to train mode
  model.train()
  for epoch in range(epochs):
      running_loss = 0
      for batch_idx, (data0, data1, data2) in enumerate(trainloader):
          anchor, positive, negative = data0, data1, data2
          anchor = Variable(anchor)
          positive = Variable(positive)
          negative = Variable(negative)
          # print('anchor', anchor.size())
          # print('positive', positive.size())
          # print('negative', negative.size())
          
          # Calculate the output of three networks
          embedded_a, embedded_p, embedded_n = model(anchor, positive, negative)
          
          # Calculate the loss
          loss = criterion(embedded_a, embedded_p, embedded_n)
          print("mini Batch Loss: {}".format(loss.data))
          
          # Zero the gradient
          optimizer.zero_grad()
          
          # Back prop and update
          loss.backward()
          optimizer.step()
          
          # print statistics
          running_loss += loss.item()

          if batch_idx % 30 == 0:
            print("Training Batch: {0} | Training Loss: {1}".format(batch_idx+1, loss.data))
            save_path = f'/content/drive/My Drive/test2/model_epoch_{epoch+1}_batch_{batch_idx+1}.pt'
            torch.save({'Batch': batch_idx, 'model_state_dict': model.state_dict()}, save_path)
            print("Training Batch: {0} | Model saved to: {1}".format(batch_idx+1, save_path))

            ''' For Validation'''
            # Change to evaluation mode
            model.eval()

            mean_accuracy = val_accuracy(model, valloader, valloader_iterator)
            print(mean_accuracy)

            # Change back to train mode
            model.train()

          # Empty the cache of CUDA  
          torch.cuda.empty_cache()

          
      print(f'[{epoch + 1}] average loss per epoch: {running_loss / len(train_loader):.3f}')
      # # save checkpoint of model
      # if epoch % 5 == 0 and epoch > 0:

      save_path = f'/content/drive/My Drive/test2/model_epoch_{epoch}.pt'
      torch.save({'epoch': epoch, 'model_state_dict': model.state_dict()}, save_path)
      print(f'Saved model checkpoint to {save_path}')

      ''' For Validation'''
      # Change to evaluation mode
      model.eval()

      mean_accuracy = val_accuracy(model, valloader, valloader_iterator)
      print(mean_accuracy)

      # Change back to train mode
      model.train()

      
  print('Finished Training')

In [118]:
trained_net = train(net, criterion, optimizer, 1, train_loader, val_loader)

================== START TRAINING ==================
mini Batch Loss: 4.412944793701172
Training Batch: 1 | Training Loss: 4.412944793701172
Training Batch: 1 | Model saved to: /content/drive/My Drive/test2/model_epoch_1_batch_1.pt
dist a: tensor([18.4125, 14.7937, 12.6985, 14.4316, 12.9494, 14.3909, 13.5844, 12.5614,
        13.7599, 17.7909, 13.2882, 19.5298, 14.3636, 13.2031, 15.4984, 16.8897,
        16.1553, 15.2223, 13.0401, 17.1888, 14.8710, 14.5714, 12.3360, 16.0529,
        13.5545, 14.7240, 14.7704, 17.2614, 14.5392, 10.7585, 13.2949, 14.2915,
        12.3717, 11.3969, 16.5193, 15.9918, 14.4647, 15.5827, 13.5296, 13.6141,
        14.3182, 14.7529, 16.3411, 14.9880, 20.2623, 15.0018, 15.0246, 15.1213,
        14.2975, 10.4766, 15.4648, 14.6889, 14.0904, 13.5552, 14.4220, 12.6184,
        14.4068, 16.6642, 16.8570, 13.0066, 16.0955, 15.2479, 15.8116, 14.5757],
       device='cuda:0'), dist b: tensor([17.5949, 17.3178, 15.9329, 15.1129, 13.9413, 16.6605, 17.4067, 15.9361,
      

In [120]:
trained_net = net

In [121]:
trained_net.eval()

DataParallel(
  (module): TripletNet(
    (embeddingnet): EmbeddingNet(
      (features): Sequential(
        (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (4): Sequential(
          (0): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (relu): ReLU(inplace=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (1): BasicBlock(
            (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1)

In [57]:
import torch
# load model from checkpoint
model_loaded = TripletNet(FeatureExtractNET())
checkpoint = torch.load('/content/drive/MyDrive/model_epoch_0.pt')
model_loaded.load_state_dict(checkpoint['model_state_dict'], strict=False)

model_loaded.to(device)
model_loaded.eval()

TripletNet(
  (embeddingnet): EmbeddingNet(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d

In [122]:
valloader_iterator = iter(val_loader)
mean_accuracy = val_accuracy(trained_net, val_loader, valloader_iterator)
print(mean_accuracy)

dist a: tensor([14.7346, 18.2229, 17.9004, 13.7015, 17.1378, 16.7035, 19.6870, 14.1521,
        16.5130, 18.5596, 21.3687, 19.8480, 21.9997, 14.3533, 16.9440, 18.4767,
        22.9334, 16.3721, 16.1076, 17.8330, 17.4193, 12.1379, 13.8147, 18.4836,
        18.2759, 22.5580, 14.2527, 19.8612, 18.2120, 16.9553, 11.7606, 14.0963,
        14.5604, 14.7805, 22.5439, 15.8438, 11.5091, 15.8298, 15.2355, 15.0780,
        16.2959, 17.9183, 17.3548, 13.3215, 25.3425, 11.0275, 19.2948, 23.6729,
        16.2255, 10.3209, 25.5373, 17.3512, 25.1248, 14.6714, 25.6110, 14.3367,
        14.2538, 22.0254, 23.9481, 14.3851, 16.0434, 16.8749, 23.4274, 21.3770],
       device='cuda:0'), dist b: tensor([31.9620, 22.1136, 27.9829, 14.0639, 20.7262, 23.3005, 23.3759, 17.6575,
        25.4295, 28.7724, 16.4912, 19.6447, 23.2085, 19.5990, 23.3683, 20.1920,
        25.5814, 31.0055, 18.6461, 24.2609, 25.0172, 26.5410, 19.5112, 22.7975,
        17.5284, 22.9672, 17.2604, 22.3534, 16.1531, 23.8159, 20.9159, 14.6986

In [123]:
print('================== START PREDICTION ==================')

# Change to evaluation mode
model_loaded.eval()

redicted_labels = np.zeros(59544)
pred_test=[]

#Predict labels 1 or 0 for each test triplet
for batch_idx, (data1, data2, data3) in enumerate(test_loader):

    data1, data2, data3 = data1.cuda(), data2.cuda(), data3.cuda()

    # wrap in torch.autograd.Variable
    data1, data2, data3 = Variable(data1), Variable(data2), Variable(data3)

    with torch.no_grad():
        # compute output and loss
        embedded_x, embedded_y, embedded_z = trained_net(data1, data2, data3)

    dist_a = F.pairwise_distance(embedded_x, embedded_y, 2)
    dist_b = F.pairwise_distance(embedded_x, embedded_z, 2)
    #print(np.squeeze(embedded_a.cpu().detach().numpy()).shape)
    

    pred_test.append(1*(dist_a <= dist_b))


    print('batch: ', batch_idx)


predicted_labels = np.hstack(pred_test)
print(predicted_labels)

#Write submisison file
df = pd.DataFrame(predicted_labels)
df.to_csv('/content/drive/MyDrive/test2/submission_1epoch.txt', index=False, header=None) #write CSV

================== START PREDICTION ==================
batch:  0
batch:  1
batch:  2
batch:  3
batch:  4
batch:  5
batch:  6
batch:  7
batch:  8
batch:  9
batch:  10
batch:  11
batch:  12
batch:  13
batch:  14
batch:  15
batch:  16
batch:  17
batch:  18
batch:  19
batch:  20
batch:  21
batch:  22
batch:  23
batch:  24
batch:  25
batch:  26
batch:  27
batch:  28
batch:  29
batch:  30
batch:  31
batch:  32
batch:  33
batch:  34
batch:  35
batch:  36
batch:  37
batch:  38
batch:  39
batch:  40
batch:  41
batch:  42
batch:  43
batch:  44
batch:  45
batch:  46
batch:  47
batch:  48
batch:  49
batch:  50
batch:  51
batch:  52
batch:  53
batch:  54
batch:  55
batch:  56
batch:  57
batch:  58
batch:  59
batch:  60
batch:  61
batch:  62
batch:  63
batch:  64
batch:  65
batch:  66
batch:  67
batch:  68
batch:  69
batch:  70
batch:  71
batch:  72
batch:  73
batch:  74
batch:  75
batch:  76
batch:  77
batch:  78
batch:  79
batch:  80
batch:  81
batch:  82
batch:  83
batch:  84
batch:  85
batch:  8

TypeError: ignored

In [136]:
pred_test_np = []
for i in range(len(pred_test)):
  pred_test_cpu = pred_test[i].cpu().detach().numpy()
  pred_test_np += list(pred_test_cpu)
len(pred_test_np)
predicted_labels = np.hstack(pred_test_np)
print(predicted_labels)

#Write submisison file
df = pd.DataFrame(predicted_labels)
df.to_csv('/content/drive/MyDrive/test2/submission_1epoch.txt', index=False, header=None) #write CSV

[1 0 0 ... 0 1 0]


In [137]:
count = len(open("/content/drive/MyDrive/test2/submission_1epoch.txt",'rU').readlines())
print(count)

59544


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: 'U' mode is deprecated
  """Entry point for launching an IPython kernel.
